Top 10 providers by total claims billed

In [0]:
SELECT 
    p.name AS provider_name,
    p.specialty,
    COUNT(c.claim_id) AS total_claims,
    SUM(c.claim_amount) AS total_billed,
    AVG(c.claim_amount) AS avg_claim_amount
FROM healthcare_cp.hcp.claims_data c
JOIN healthcare_cp.hcp.providers_data p ON c.provider_id = p.provider_id
GROUP BY p.provider_id, p.name, p.specialty
ORDER BY total_billed DESC
LIMIT 10;

Top 10 providers by total revenue (paid_amount)

In [0]:
SELECT 
    p.name AS provider_name,
    p.specialty,
    p.organization,
    COUNT(c.claim_id) AS total_claims,
    SUM(c.paid_amount) AS total_revenue,
    SUM(c.claim_amount) AS total_billed,
    ROUND((SUM(c.paid_amount) / SUM(c.claim_amount) * 100), 2) AS payment_rate_pct
FROM healthcare_cp.hcp.claims_data c
JOIN healthcare_cp.hcp.providers_data p ON c.provider_id = p.provider_id
WHERE c.paid_amount > 0
GROUP BY p.provider_id, p.name, p.specialty, p.organization
ORDER BY total_revenue DESC
LIMIT 10;

Average claim amount per specialty

In [0]:
SELECT 
    p.specialty,
    COUNT(c.claim_id) AS total_claims,
    ROUND(AVG(CAST(c.claim_amount AS FLOAT)), 2) AS avg_claim_amount,
    ROUND(STDDEV(CAST(c.claim_amount AS FLOAT)), 2) AS std_dev_amount,
    MIN(CAST(c.claim_amount AS FLOAT)) AS min_amount,
    MAX(CAST(c.claim_amount AS FLOAT)) AS max_amount,
    ROUND(MEDIAN(CAST(c.claim_amount AS FLOAT)), 2) AS median_amount
FROM healthcare_cp.hcp.claims_data c
JOIN healthcare_cp.hcp.providers_data p ON c.provider_id = p.provider_id
GROUP BY p.specialty
ORDER BY avg_claim_amount DESC;

Flag providers with >2 std dev above avg claim count (Z-score)

In [0]:
WITH provider_stats AS (
    SELECT 
        provider_id,
        COUNT(claim_id) AS claim_count
    FROM healthcare_cp.hcp.claims_data
    GROUP BY provider_id
),
population_stats AS (
    SELECT 
        AVG(claim_count) AS avg_claims,
        STDDEV(claim_count) AS std_dev_claims
    FROM provider_stats
),
z_scores AS (
    SELECT 
        ps.provider_id,
        ps.claim_count,
        pop.avg_claims,
        pop.std_dev_claims,
        (ps.claim_count - pop.avg_claims) / pop.std_dev_claims AS z_score
    FROM provider_stats ps
    CROSS JOIN population_stats pop
)
SELECT 
    p.name AS provider_name,
    p.specialty,
    p.years_of_experience,
    z.claim_count,
    ROUND(z.avg_claims, 2) AS avg_claims,
    ROUND(z.z_score, 3) AS z_score,
    CASE 
        WHEN z.z_score > 2 THEN 'HIGH OUTLIER'
        WHEN z.z_score < -2 THEN 'LOW OUTLIER'
        ELSE 'NORMAL'
    END AS outlier_status
FROM z_scores z
JOIN healthcare_cp.hcp.providers_data p ON z.provider_id = p.provider_id
WHERE z.z_score > 2
ORDER BY z.z_score DESC;

Providers with >20% claims denied

In [0]:
SELECT 
    p.name AS provider_name,
    p.specialty,
    p.organization,
    COUNT(c.claim_id) AS total_claims,
    SUM(CASE WHEN c.claim_status = 'Denied' THEN 1 ELSE 0 END) AS denied_claims,
    ROUND((SUM(CASE WHEN c.claim_status = 'Denied' THEN 1 ELSE 0 END) * 100.0 / COUNT(c.claim_id)), 2) AS denial_rate_pct,
    SUM(CASE WHEN c.claim_status = 'Denied' THEN CAST(c.claim_amount AS FLOAT) ELSE 0 END) AS denied_amount
FROM healthcare_cp.hcp.claims_data c
JOIN healthcare_cp.hcp.providers_data p ON c.provider_id = p.provider_id
GROUP BY p.provider_id, p.name, p.specialty, p.organization
HAVING (SUM(CASE WHEN c.claim_status = 'Denied' THEN 1 ELSE 0 END) * 100.0 / COUNT(c.claim_id)) > 20
ORDER BY denial_rate_pct DESC;

New providers (less than 1 year experience) with high claim volumes

In [0]:
WITH new_provider_claims AS (
    SELECT 
        p.provider_id,
        p.name,
        p.specialty,
        p.years_of_experience,
        COUNT(c.claim_id) AS claim_count,
        SUM(c.claim_amount) AS total_billed,
        AVG(COUNT(c.claim_id)) OVER() AS avg_claims_all_providers
    FROM healthcare_cp.hcp.providers_data p
    LEFT JOIN healthcare_cp.hcp.claims_data c ON p.provider_id = c.provider_id
    WHERE p.years_of_experience < 1
    GROUP BY p.provider_id, p.name, p.specialty, p.years_of_experience
)
SELECT 
    name AS provider_name,
    specialty,
    years_of_experience,
    claim_count,
    total_billed,
    ROUND(avg_claims_all_providers, 0) AS avg_claims_benchmark,
    ROUND((claim_count / avg_claims_all_providers), 2) AS volume_ratio
FROM new_provider_claims
WHERE claim_count > avg_claims_all_providers
ORDER BY claim_count DESC;

Top specialties by emergency claim frequency

In [0]:
SELECT 
    p.specialty,
    COUNT(c.claim_id) AS total_claims,
    SUM(CASE WHEN c.is_emergency = true THEN 1 ELSE 0 END) AS emergency_claims,
    ROUND((SUM(CASE WHEN c.is_emergency = true THEN 1 ELSE 0 END) * 100.0 / COUNT(c.claim_id)), 2) AS emergency_rate_pct,
    ROUND(AVG(CASE WHEN c.is_emergency = true THEN c.claim_amount END), 2) AS avg_emergency_amount,
    COUNT(DISTINCT c.provider_id) AS providers_in_specialty
FROM healthcare_cp.hcp.claims_data c
JOIN healthcare_cp.hcp.providers_data p ON c.provider_id = p.provider_id
GROUP BY p.specialty
ORDER BY emergency_rate_pct DESC;